In [ ]:
import os
from pprint import pprint
if '_fixed' not in locals():
    _fixed = os.chdir(os.path.dirname(os.getcwd()))
from pathlib import Path
import json, yaml
import inspect
import jsonschema
from tabulate import tabulate
from jsonschema import validate, ValidationError
from tqdm.notebook import tqdm
from ludwig.util import PromptTemplate, repo_root
from ludwig.tictactoe import TakeTheMiddle
from ludwig.util import vllm_Client
from ludwig.util import extract_code_blocks
from ludwig.jsonutils import unflatten, flatten


In [2]:
# jobdir = Path(r'local_data/tree-build-ChessPuzzle-')
jobdir = Path(r'local_data/tree-build-ChessPuzzle-1f77_250808-174050') # active
jobdir = Path(r'local_data/tree-build-ChessPuzzle-1f77_250809-234455') # fen
jobdir = Path(r'local_data/tree-build-ChessPuzzle-1f77_250809-230620') # pgn
jobdir = Path(r'local_data/tree-build-ChessPuzzle-c635_250923-154828') # fen
# jobdir = Path(r'local_data/tree-build-TakeTheMiddle-1f77_250809-003845') # moves
# jobdir = Path(r'local_data/tree-build-TakeTheMiddle-1f77_250809-005349') # compact
# jobdir = Path(r'local_data/tree-build-TakeTheMiddle-8fc9_250813-230409')
path = jobdir.joinpath('log.jsonl')
jobdir.exists()

True

In [3]:
full = [unflatten(json.loads(line)) for line in path.read_text().splitlines()]
len(full)

10

In [4]:
item = full[0]
print(tabulate(item.items()))

----------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
item['table'].keys()

dict_keys(['index', 'seed', 'task', 'context', 'rep', 'expand_code', 'extract_code', 'extract_approach', 'evaluate_code', 'evaluate_approach', 'chat'])

In [6]:
print(item['table']['chat'][0]['content'])

We will solve the following task using a structured tree-search framework.

**Task Description:**
Can you solve this chess puzzle? You will be given a board position and you must find the best move for the side to move. 

**Example Problem:**
```
White just played Rxa7. Given the resulting board position:

FEN: r5k1/Rp3p1p/2b2qp1/3pr3/8/4P2P/2PN1PP1/Q3K2R b K - 0 19

What is the best move for black? Answer using the UCI or SAN format.
```

Your first task is to design an optimal state representation for this kind of problem. The representation must be:
1.  **High-Fidelity:** It must capture all details necessary to solve the task unambiguously.
2.  **Efficient:** It should contain minimal redundancy and exclude irrelevant "nuisance" details.
3.  **Conducive to Search:** The structure should make it easy to implement the search functions (`expand`, `extract`, `evaluate`) which are crucial for the tree search algorithm.

Define this representation as a JSON schema, including clear descri

In [ ]:
for item in full:
    # pprint(item['table']['rep'])
    rep = item['table']['rep']
    print(yaml.dump(rep, sort_keys=False, allow_uniencode=True, default_flow_style=None))
    print('*'*100)
    print()

$schema: http://json-schema.org/draft-07/schema#
title: ChessPuzzleState
description: High‑fidelity, concise representation of a position for puzzle solving.
type: object
required: [board, turn, activeColor, hash, sideCastling, enPassant, halfMoveClock,
  fullMoveNumber]
properties:
  board:
    description: A 0‑indexed 8×8 array of piece objects.
    type: array
    minItems: 8
    items:
      type: array
      minItems: 8
      items:
        type: object
        properties:
          type:
            description: 'Piece type: ''p'',''n'',''b'',''r'',''q'',''k'' for black
              and uppercase for white.'
            type: [string, 'null']
            pattern: ^[pnbrqkPNBRQK]$
          color:
            description: 'Piece color: ''w'' or ''b''.'
            type: [string, 'null']
            pattern: ^[wb]$
        additionalProperties: false
        required: [type, color]
  turn:
    description: 'Side to move: 0 for White, 1 for Black.'
    type: integer
    enum: [0, 1

In [30]:

_schema_bad_keys = {'uniqueItems', '$schema', 'pattern', 'examples', 'example'}
class SchemaInvalidError(ValueError):
    pass

def filter_schema(data):
    if isinstance(data, dict):
        if data.get('type') == 'array' and 'items' not in data:
            raise SchemaInvalidError("Array schema must have 'items' key")
        if 'items' in data and isinstance(data['items'], list):
            assert len(data['items']) == 1
            data['items'] = data['items'][0]
        return {key: filter_schema(val) for key, val in data.items() if key not in _schema_bad_keys}
    if isinstance(data, list):
        return [filter_schema(item) for item in data]
    return data

def publish(fn):
    if fn is None:
        return fn
    return inspect.getsource(fn).replace(fn.__name__, 'formalize')

In [15]:
from ludwig.util import vllm_Client, SAIA_Client
# client = SAIA_Client.connect()
client = vllm_Client('8001')
client.prepare()
client.ident

'openai/gpt-oss-20b'

In [ ]:
outpath = jobdir.joinpath('log-formal.json')
if outpath.exists():
    raise RuntimeError(f"{outpath} exists")

RuntimeError: local_data\tree-build-ChessPuzzle-c635_250923-154828\log-formal.json exists

In [233]:
index = 9
item = full[index]
rep = filter_schema(item['table']['rep'])
rep

{'title': 'Chess Position',
 'description': 'Minimal, complete representation of a chess position for tree‑search.',
 'type': 'object',
 'required': ['board',
  'turn',
  'halfmove',
  'fullmove',
  'castling',
  'enpassant'],
 'properties': {'board': {'description': 'Mapping from square identifiers ("a1" … "h8") to piece codes.\nPiece codes: K,Q,R,B,N,P for White; k,q,r,b,n,p for Black.\nSquares not present are empty.',
   'type': 'object',
   'additionalProperties': {'type': 'string'},
   'minProperties': 0},
  'turn': {'description': 'Side to move: "w" for White, "b" for Black.',
   'type': 'string',
   'enum': ['w', 'b']},
  'castling': {'description': 'String of castling rights: any subset of \'KQkq\', or "" if none.',
   'type': 'string'},
  'enpassant': {'description': 'Square where an en‑passant capture is possible, or "-" if none.',
   'type': 'string'},
  'halfmove': {'description': 'Half‑move clock (pawn move or capture since last reset).',
   'type': 'integer',
   'minimum'

In [234]:
chat = client.begin_chat(f'Give me a random state of a Chess game using this representation?\n{json.dumps(rep, indent=2)}')
resp = client.step(chat, grammar=rep, max_tokens=4096)
raw = chat[-1]['content']
example = json.loads(raw)
example

{'board': {'g1': 'K',
  'd1': 'Q',
  'a1': 'R',
  'f1': 'R',
  'c1': 'B',
  'f4': 'B',
  'e4': 'N',
  'a2': 'P',
  'b2': 'P',
  'c2': 'P',
  'd4': 'P',
  'f2': 'P',
  'g2': 'P',
  'h2': 'P',
  'g8': 'k',
  'd8': 'q',
  'a8': 'r',
  'f8': 'r',
  'c8': 'b',
  'c3': 'b',
  'f6': 'n',
  'a7': 'p',
  'b7': 'p',
  'c7': 'p',
  'd5': 'p',
  'e7': 'p',
  'f7': 'p',
  'g7': 'p',
  'h7': 'p'},
 'turn': 'b',
 'castling': '',
 'enpassant': '-',
 'halfmove': 4,
 'fullmove': 12}

In [235]:
example_state = 'r5k1/pp3p1p/2b2qp1/3pr3/8/4P2P/R1PN1PP1/Q3K2R w K - 0 19'
enctmpl = PromptTemplate('formalization-gt')
print(enctmpl.fill(state=example_state, schema=json.dumps(rep, indent=2)))

Implement a python function that is given a state representation of the form **INPUT** and converts it into a state representation that matches the following JSON schema **OUTPUT**. Call the function `encode`.

Example **INPUT**:
```
r5k1/pp3p1p/2b2qp1/3pr3/8/4P2P/R1PN1PP1/Q3K2R w K - 0 19
```

**OUTPUT** schema:
```json
{
  "title": "Chess Position",
  "description": "Minimal, complete representation of a chess position for tree\u2011search.",
  "type": "object",
  "required": [
    "board",
    "turn",
    "halfmove",
    "fullmove",
    "castling",
    "enpassant"
  ],
  "properties": {
    "board": {
      "description": "Mapping from square identifiers (\"a1\" \u2026 \"h8\") to piece codes.\nPiece codes: K,Q,R,B,N,P for White; k,q,r,b,n,p for Black.\nSquares not present are empty.",
      "type": "object",
      "additionalProperties": {
        "type": "string"
      },
      "minProperties": 0
    },
    "turn": {
      "description": "Side to move: \"w\" for White, \"b\" for Bl

In [236]:
print(rep)

{'title': 'Chess Position', 'description': 'Minimal, complete representation of a chess position for tree‑search.', 'type': 'object', 'required': ['board', 'turn', 'halfmove', 'fullmove', 'castling', 'enpassant'], 'properties': {'board': {'description': 'Mapping from square identifiers ("a1" … "h8") to piece codes.\nPiece codes: K,Q,R,B,N,P for White; k,q,r,b,n,p for Black.\nSquares not present are empty.', 'type': 'object', 'additionalProperties': {'type': 'string'}, 'minProperties': 0}, 'turn': {'description': 'Side to move: "w" for White, "b" for Black.', 'type': 'string', 'enum': ['w', 'b']}, 'castling': {'description': 'String of castling rights: any subset of \'KQkq\', or "" if none.', 'type': 'string'}, 'enpassant': {'description': 'Square where an en‑passant capture is possible, or "-" if none.', 'type': 'string'}, 'halfmove': {'description': 'Half‑move clock (pawn move or capture since last reset).', 'type': 'integer', 'minimum': 0}, 'fullmove': {'description': 'Full‑move numb

In [237]:
print(example)

{'board': {'g1': 'K', 'd1': 'Q', 'a1': 'R', 'f1': 'R', 'c1': 'B', 'f4': 'B', 'e4': 'N', 'a2': 'P', 'b2': 'P', 'c2': 'P', 'd4': 'P', 'f2': 'P', 'g2': 'P', 'h2': 'P', 'g8': 'k', 'd8': 'q', 'a8': 'r', 'f8': 'r', 'c8': 'b', 'c3': 'b', 'f6': 'n', 'a7': 'p', 'b7': 'p', 'c7': 'p', 'd5': 'p', 'e7': 'p', 'f7': 'p', 'g7': 'p', 'h7': 'p'}, 'turn': 'b', 'castling': '', 'enpassant': '-', 'halfmove': 4, 'fullmove': 12}


In [246]:
gt = [
    {
        'rep': {'title': 'ChessPuzzleState', 'description': 'High‑fidelity, concise representation of a position for puzzle solving.', 'type': 'object', 'required': ['board', 'turn', 'activeColor', 'hash', 'sideCastling', 'enPassant', 'halfMoveClock', 'fullMoveNumber'], 'properties': {'board': {'description': 'A 0‑indexed 8×8 array of piece objects.', 'type': 'array', 'minItems': 8, 'items': {'type': 'array', 'minItems': 8, 'items': {'type': 'object', 'properties': {'type': {'description': "Piece type: 'p','n','b','r','q','k' for black and uppercase for white.", 'type': ['string', 'null']}, 'color': {'description': "Piece color: 'w' or 'b'.", 'type': ['string', 'null']}}, 'additionalProperties': False, 'required': ['type', 'color']}}}, 'turn': {'description': 'Side to move: 0 for White, 1 for Black.', 'type': 'integer', 'enum': [0, 1]}, 'activeColor': {'description': 'The side that just moved (the owner of the last move).', 'type': 'string', 'enum': ['white', 'black']}, 'hash': {'description': 'Zobrist hash of the current position, used for transposition table lookup.', 'type': 'string'}, 'sideCastling': {'description': 'Castling availability for each side.', 'type': 'object', 'properties': {'whiteK': {'type': 'boolean'}, 'whiteQ': {'type': 'boolean'}, 'blackK': {'type': 'boolean'}, 'blackQ': {'type': 'boolean'}}, 'additionalProperties': False}, 'enPassant': {'description': 'Target square for a en‑passant capture or null if none.', 'type': ['string', 'null']}, 'halfMoveClock': {'description': 'Number of half‑moves since the last capture or pawn move (for the 50‑move draw rule).', 'type': 'integer', 'minimum': 0}, 'fullMoveNumber': {'description': "Move count starting from 1, incremented after Black's move.", 'type': 'integer', 'minimum': 1}, 'pawnsWhite': {'description': 'Bitboard representation of white pawns.', 'type': 'string'}, 'pawnsBlack': {'description': 'Bitboard representation of black pawns.', 'type': 'string'}, 'unknownProperties': {'description': 'Optional map for fast access by external modules. Should not be mutated by the search engine.', 'type': 'object', 'additionalProperties': True}}, 'additionalProperties': False},
        'example': {'board': [[{'type': 'r', 'color': 'b'}, {'type': 'n', 'color': 'b'}, {'type': 'b', 'color': 'b'}, {'type': 'q', 'color': 'b'}, {'type': None, 'color': None}, {'type': 'r', 'color': 'b'}, {'type': 'k', 'color': 'b'}, {'type': 'b', 'color': 'b'}], [{'type': 'p', 'color': 'b'}, {'type': 'p', 'color': 'b'}, {'type': 'p', 'color': 'b'}, {'type': 'p', 'color': 'b'}, {'type': 'b', 'color': 'b'}, {'type': 'p', 'color': 'b'}, {'type': 'p', 'color': 'b'}, {'type': 'p', 'color': 'b'}], [{'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': 'n', 'color': 'b'}, {'type': None, 'color': None}, {'type': None, 'color': None}], [{'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': 'p', 'color': 'b'}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}], [{'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': 'p', 'color': 'w'}, {'type': 'n', 'color': 'w'}, {'type': None, 'color': None}, {'type': None, 'color': None}], [{'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}, {'type': None, 'color': None}], [{'type': 'p', 'color': 'w'}, {'type': 'p', 'color': 'w'}, {'type': 'p', 'color': 'w'}, {'type': 'p', 'color': 'w'}, {'type': 'b', 'color': 'w'}, {'type': 'p', 'color': 'w'}, {'type': 'p', 'color': 'w'}, {'type': 'p', 'color': 'w'}], [{'type': 'r', 'color': 'w'}, {'type': 'n', 'color': 'w'}, {'type': 'b', 'color': 'w'}, {'type': 'q', 'color': 'w'}, {'type': None, 'color': None}, {'type': 'r', 'color': 'w'}, {'type': 'k', 'color': 'w'}, {'type': None, 'color': None}]], 'turn': 1, 'activeColor': 'white', 'hash': '4e3b1d0f6a9c2b8c7d5e4f3a2b1c0d9e8f7a6b5c4d3e2f1g0h', 'sideCastling': {'whiteK': False, 'whiteQ': False, 'blackK': True, 'blackQ': True}, 'enPassant': 'e3', 'halfMoveClock': 0, 'fullMoveNumber': 1, 'pawnsWhite': '0000000000000000100000000000001100001000000000000000000000000000', 'pawnsBlack': '0000000010000000000010000000001001000000000000000000000000000000', 'unknownProperties': {}},
        'decode': '''import json
from typing import Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a JSON chess state representation into its FEN string.

    This function processes a dictionary containing detailed information about a
    chess position—such as piece placement, turn, castling rights, and move
    clocks—and translates it into the standard Forsyth-Edwards Notation (FEN).

    Args:
        state: A dictionary representing the chess position, conforming to the
               specified input schema.

    Returns:
        The FEN string corresponding to the input state.
    """
    
    # 1. Piece Placement
    board_fen_parts = []
    for row in state['board']:
        empty_squares = 0
        rank_fen = ""
        for piece in row:
            if piece['type'] is None:
                empty_squares += 1
            else:
                if empty_squares > 0:
                    rank_fen += str(empty_squares)
                empty_squares = 0
                
                piece_char = piece['type']
                if piece['color'] == 'w':
                    rank_fen += piece_char.upper()
                else: # 'b'
                    rank_fen += piece_char.lower()
        
        if empty_squares > 0:
            rank_fen += str(empty_squares)
            
        board_fen_parts.append(rank_fen)
    
    piece_placement = "/".join(board_fen_parts)

    # 2. Active Color
    active_color = 'w' if state['turn'] == 0 else 'b'

    # 3. Castling Availability
    castling = ""
    side_castling = state['sideCastling']
    if side_castling.get('whiteK'):
        castling += 'K'
    if side_castling.get('whiteQ'):
        castling += 'Q'
    if side_castling.get('blackK'):
        castling += 'k'
    if side_castling.get('blackQ'):
        castling += 'q'
    
    if not castling:
        castling = '-'

    # 4. En Passant Target Square
    en_passant = state['enPassant'] if state['enPassant'] is not None else '-'

    # 5. Halfmove Clock
    halfmove_clock = str(state['halfMoveClock'])

    # 6. Fullmove Number
    fullmove_number = str(state['fullMoveNumber'])

    # Assemble the FEN string
    fen_parts = [
        piece_placement,
        active_color,
        castling,
        en_passant,
        halfmove_clock,
        fullmove_number
    ]
    
    return " ".join(fen_parts)''',
        'encode': '''def encode(fen_string: str) -> dict:
    """
    Converts a chess position from a FEN string to a dictionary
    that matches the specified JSON schema.

    Args:
        fen_string: The FEN string representing the chess position.

    Returns:
        A dictionary representing the state of the chess puzzle.
    """
    
    # 1. Split the FEN string into its 6 components
    (
        piece_placement,
        active_color,
        castling,
        en_passant,
        halfmove_clock,
        fullmove_number,
    ) = fen_string.split()

    # 2. Process the board (piece placement)
    board = []
    for rank_str in piece_placement.split('/'):
        row = []
        for char in rank_str:
            if char.isdigit():
                # Add empty squares for numeric values
                row.extend([{"type": None, "color": None}] * int(char))
            else:
                # Add the piece object
                color = 'w' if char.isupper() else 'b'
                piece_type = char.lower()
                row.append({"type": piece_type, "color": color})
        board.append(row)

    # 3. Process castling availability
    side_castling = {
        "whiteK": 'K' in castling,
        "whiteQ": 'Q' in castling,
        "blackK": 'k' in castling,
        "blackQ": 'q' in castling,
    }

    # 4. Assemble the final state dictionary
    state = {
        "board": board,
        "turn": 0 if active_color == 'w' else 1,
        "activeColor": "white" if active_color == 'w' else "black",
        "hash": "",  # Zobrist hash calculation is complex and not specified.
        "sideCastling": side_castling,
        "enPassant": None if en_passant == '-' else en_passant,
        "halfMoveClock": int(halfmove_clock),
        "fullMoveNumber": int(fullmove_number),
    }

    return state'''
    },
    {
        'rep': {'title': 'ChessPosition', 'description': 'A minimal, high‑fidelity representation of a chess position suitable for tree‑search.', 'type': 'object', 'required': ['board', 'side', 'castling', 'en_passant', 'halfmove_clock', 'fullmove_number'], 'properties': {'board': {'description': '8x8 array of strings representing pieces or empty squares. Row 0 is the 8th rank, row 7 is the 1st rank.', 'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'type': 'string', 'enum': ['', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k']}}}, 'side': {'description': "Side to move. 'w' for white, 'b' for black.", 'type': 'string', 'enum': ['w', 'b']}, 'castling': {'description': "String containing any of the letters KQkq, or '-' if no castling rights remain.", 'type': 'string'}, 'en_passant': {'description': "Square where a capture en passant is possible, e.g. 'e6'. Use '-' if none.", 'type': 'string'}, 'halfmove_clock': {'description': 'Number of half‑moves since the last pawn advance or capture (used for the fifty‑move rule).', 'type': 'integer', 'minimum': 0}, 'fullmove_number': {'description': 'The number of the full move. Increments after Black’s move.', 'type': 'integer', 'minimum': 1}, 'move_history': {'description': 'Optional ordered list of moves made so far in UCI format. Useful for debugging but not required for a single position.', 'type': 'array', 'items': {'type': 'string'}}}, 'additionalProperties': False}, 
        'example': {'board': [['r', 'n', 'b', 'q', 'k', '', '', 'r'], ['p', 'p', 'p', 'p', '', 'p', 'p', 'p'], ['', '', '', '', '', '', '', ''], ['', '', '', 'n', '', '', '', ''], ['', '', '', '', 'P', '', '', ''], ['', '', '', '', '', 'N', '', ''], ['P', 'P', 'P', 'P', '', 'P', 'P', 'P'], ['R', 'N', 'B', 'Q', 'K', 'B', '', 'R']], 'side': 'w', 'castling': 'KQkq', 'en_passant': '-', 'halfmove_clock': 2, 'fullmove_number': 2, 'move_history': ['e2e4', 'c7c5', 'g1f3', 'b8c6']},
        'decode': '''import json
from typing import Dict, List, Union

def decode(state: Dict[str, Union[List[List[str]], str, int]]) -> str:
    """
    Converts a JSON chess position representation into a FEN string.

    Args:
        state: A dictionary representing the chess position, conforming to the
               specified input schema.

    Returns:
        The FEN (Forsyth-Edwards Notation) string for the position.
    """
    # 1. Piece Placement
    board_fen_parts = []
    for row in state['board']:
        rank_fen = ""
        empty_squares = 0
        for piece in row:
            if piece == "":
                empty_squares += 1
            else:
                if empty_squares > 0:
                    rank_fen += str(empty_squares)
                    empty_squares = 0
                rank_fen += piece
        # Append count for any trailing empty squares
        if empty_squares > 0:
            rank_fen += str(empty_squares)
        board_fen_parts.append(rank_fen)
    
    board_part = "/".join(board_fen_parts)

    # 2-6. Other FEN fields from the input state
    side_part = state['side']
    castling_part = state['castling']
    en_passant_part = state['en_passant']
    halfmove_clock_part = str(state['halfmove_clock'])
    fullmove_number_part = str(state['fullmove_number'])

    # Assemble the final FEN string
    fen_string = " ".join([
        board_part,
        side_part,
        castling_part,
        en_passant_part,
        halfmove_clock_part,
        fullmove_number_part
    ])
    
    return fen_string''',
        'encode': '''def encode(fen_string: str):
    """
    Converts a FEN string representation of a chess position into a
    dictionary that matches the specified ChessPosition JSON schema.

    Args:
        fen_string: A standard FEN string.

    Returns:
        A dictionary representing the chess position.
    """
    
    # Split the FEN string into its six component fields
    (
        piece_placement,
        side_to_move,
        castling,
        en_passant,
        halfmove_clock,
        fullmove_number,
    ) = fen_string.split(" ")

    # --- 1. Process the board's piece placement ---
    board = []
    # Iterate over each rank (separated by '/')
    for rank_str in piece_placement.split("/"):
        rank = []
        # Iterate over each character in the rank string
        for char in rank_str:
            if char.isdigit():
                # If it's a digit, add that many empty squares
                rank.extend([""] * int(char))
            else:
                # If it's a letter, add the piece
                rank.append(char)
        board.append(rank)

    # --- 2. Assemble the final state dictionary ---
    chess_position = {
        "board": board,
        "side": side_to_move,
        "castling": castling,
        "en_passant": en_passant,
        "halfmove_clock": int(halfmove_clock),
        "fullmove_number": int(fullmove_number),
    }

    return chess_position'''
    },
    {
        'rep': {'title': 'Chess State Representation for Tree‑Search', 'description': 'A compact, high‑fidelity representation of a chess position suitable for tree‑search algorithms (move generation, evaluation, expansion).  All fields are required; missing information would prevent unambiguous solving.', 'type': 'object', 'required': ['board', 'sideToMove', 'castling', 'enPassantTarget', 'halfmoveClock', 'fullmoveNumber'], 'properties': {'board': {'description': "8×8 array that maps each square to a piece code or null.  The first dimension is the rank (index 0 -> rank 8, index 7 -> rank 1).  The second dimension is the file (index 0 -> file 'a', index 7 -> file 'h').", 'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'anyOf': [{'type': 'null'}, {'type': 'string', 'enum': ['P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k'], 'description': 'Uppercase = White, lowercase = Black.'}]}}}, 'sideToMove': {'type': 'string', 'enum': ['w', 'b'], 'description': "The colour whose turn it is.  'w' for White, 'b' for Black."}, 'castling': {'type': 'string', 'description': 'Castling availability in standard notation.  Empty ("" or "-") means no castling rights.'}, 'enPassantTarget': {'type': 'string', 'description': 'Target square where an en‑passant capture is possible.  "-" indicates none.'}, 'halfmoveClock': {'type': 'integer', 'minimum': 0, 'description': 'Number of half‑moves since the last pawn advance or capture (for the 50‑move draw rule).'}, 'fullmoveNumber': {'type': 'integer', 'minimum': 1, 'description': 'The full move number.  Incremented after each Black move.'}}, 'additionalProperties': False},
        'example': {'board': [['r', None, 'b', None, 'k', None, None, 'r'], ['p', 'p', 'p', None, None, 'p', 'p', 'p'], [None, None, None, None, None, None, None, None], [None, None, None, None, None, 'n', None, None], [None, None, None, None, 'P', None, None, None], [None, None, None, None, None, None, None, None], ['P', 'P', 'P', 'P', None, 'P', 'P', 'P'], ['R', 'N', 'B', 'Q', 'K', 'B', None, 'R']], 'sideToMove': 'w', 'castling': 'KQkq', 'enPassantTarget': '-', 'halfmoveClock': 4, 'fullmoveNumber': 10},
        'decode': '''import json
from typing import Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a JSON-like chess state representation into a FEN string.

    Args:
        state: A dictionary representing the chess position, following the
               specified input schema.

    Returns:
        The FEN string for the given chess position.
    """
    # 1. Piece Placement
    board_fen_ranks = []
    for rank in state['board']:
        rank_fen = ""
        empty_squares = 0
        for piece in rank:
            if piece is None:
                empty_squares += 1
            else:
                if empty_squares > 0:
                    rank_fen += str(empty_squares)
                    empty_squares = 0
                rank_fen += piece
        if empty_squares > 0:
            rank_fen += str(empty_squares)
        board_fen_ranks.append(rank_fen)
    board_part = "/".join(board_fen_ranks)

    # 2. Active Color
    side_to_move_part = state['sideToMove']

    # 3. Castling Availability
    # Use '-' if the input string is empty, otherwise use the provided string.
    castling_part = state['castling'] if state['castling'] else "-"

    # 4. En Passant Target Square
    en_passant_part = state['enPassantTarget']

    # 5. Halfmove Clock
    halfmove_part = str(state['halfmoveClock'])

    # 6. Fullmove Number
    fullmove_part = str(state['fullmoveNumber'])

    # Assemble all parts of the FEN string
    fen_parts = [
        board_part,
        side_to_move_part,
        castling_part,
        en_passant_part,
        halfmove_part,
        fullmove_part
    ]
    
    return " ".join(fen_parts)''',
        'encode': '''def encode(fen_string: str) -> dict:
    """
    Converts a FEN string into a JSON-compatible dictionary representation.

    Args:
        fen_string: A string representing a chess position in FEN format.

    Returns:
        A dictionary matching the specified chess state JSON schema.
    """
    # Split the FEN string into its six component fields.
    parts = fen_string.split()
    piece_placement, side_to_move, castling, en_passant, halfmove, fullmove = parts

    ## Board Representation
    # Process the piece placement field to create the 8x8 board array.
    board = []
    ranks = piece_placement.split('/')
    for rank_str in ranks:
        rank_list = []
        for char in rank_str:
            if char.isdigit():
                # Add null for empty squares indicated by numbers.
                rank_list.extend([None] * int(char))
            else:
                # Add the piece character.
                rank_list.append(char)
        board.append(rank_list)

    ## Construct the Final Dictionary
    # Assemble the final state dictionary from the parsed FEN components.
    state = {
        "board": board,
        "sideToMove": side_to_move,
        "castling": castling,
        "enPassantTarget": en_passant,
        "halfmoveClock": int(halfmove),
        "fullmoveNumber": int(fullmove)
    }
    return state'''
    },
    {
        'rep': {'title': 'ChessPuzzleState', 'description': 'Minimal, high‑fidelity representation of a chess puzzle state.', 'type': 'object', 'required': ['fen', 'sideToMove', 'activePieces', 'castlingRights', 'enPassantTarget', 'halfmoveClock', 'fullmoveNumber'], 'properties': {'fen': {'type': 'string', 'description': 'Standard FEN string of the board.'}, 'sideToMove': {'type': 'string', 'enum': ['w', 'b'], 'description': 'Side that has the next move.'}, 'activePieces': {'type': 'object', 'properties': {'w': {'type': 'object', 'properties': {'K': {'type': 'integer', 'minimum': 0, 'maximum': 1}, 'Q': {'type': 'integer', 'minimum': 0}, 'R': {'type': 'integer', 'minimum': 0}, 'B': {'type': 'integer', 'minimum': 0}, 'N': {'type': 'integer', 'minimum': 0}, 'P': {'type': 'integer', 'minimum': 0}}, 'additionalProperties': False}, 'b': {'type': 'object', 'properties': {'K': {'type': 'integer', 'minimum': 0, 'maximum': 1}, 'Q': {'type': 'integer', 'minimum': 0}, 'R': {'type': 'integer', 'minimum': 0}, 'B': {'type': 'integer', 'minimum': 0}, 'N': {'type': 'integer', 'minimum': 0}, 'P': {'type': 'integer', 'minimum': 0}}, 'additionalProperties': False}}, 'additionalProperties': False}, 'castlingRights': {'type': 'string', 'description': 'Castling availability per FEN (e.g., "KQkq" or "-" for none).'}, 'enPassantTarget': {'type': 'string', 'description': "En‑passant target square or '-' if none."}, 'halfmoveClock': {'type': 'integer', 'minimum': 0, 'description': 'Half‑move counter for the 50‑move rule.'}, 'fullmoveNumber': {'type': 'integer', 'minimum': 1, 'description': 'Full‑move number counter.'}}, 'additionalProperties': False},
        'example': {'fen': 'r1bqkbnr/pppp1ppp/2n5/4p3/2P5/5N2/PP1PPPPP/RNBQKB1R w KQkq - 0 4', 'sideToMove': 'w', 'activePieces': {'w': {'K': 1, 'Q': 1, 'R': 2, 'B': 2, 'N': 2, 'P': 8}, 'b': {'K': 1, 'Q': 1, 'R': 2, 'B': 1, 'N': 2, 'P': 8}}, 'castlingRights': 'KQkq', 'enPassantTarget': '-', 'halfmoveClock': 0, 'fullmoveNumber': 4},
        'decode': '''from typing import Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a chess puzzle state from a dictionary representation to a
    standard FEN string.

    Args:
        state: A dictionary conforming to the ChessPuzzleState schema,
               containing all necessary components of a chess position.

    Returns:
        The full FEN string for the given state.
    """
    # Extract the board layout from the 'fen' field. This handles cases
    # where the field might contain the full FEN or just the piece placement.
    board_layout = state['fen'].split(' ')[0]

    # Assemble the full FEN string from its constituent parts.
    fen_string = " ".join([
        board_layout,
        state['sideToMove'],
        state['castlingRights'],
        state['enPassantTarget'],
        str(state['halfmoveClock']),
        str(state['fullmoveNumber'])
    ])

    return fen_string''',
        'encode': '''def encode(fen_string: str) -> dict:
    """
    Converts a FEN string into a structured dictionary matching the specified schema.

    This function parses a standard FEN string, which represents a chess position,
    and transforms it into a detailed dictionary format. This includes counting
    all active pieces for both white and black.

    Args:
        fen_string: A standard Forsyth-Edwards Notation (FEN) string.

    Returns:
        A dictionary representing the detailed state of the chess puzzle.
        
    Raises:
        ValueError: If the input FEN string is malformed and does not contain
                    the expected six space-separated fields.
    """
    # Split the FEN string into its six standard components.
    try:
        (piece_placement, side_to_move, castling_rights, 
         en_passant_target, halfmove_clock, fullmove_number) = fen_string.split()
    except ValueError:
        raise ValueError("Invalid FEN string: must contain 6 space-separated fields.")

    # Initialize a structure to hold the counts of active pieces for each side.
    # The schema requires all piece types, even if the count is zero.
    active_pieces = {
        'w': {'K': 0, 'Q': 0, 'R': 0, 'B': 0, 'N': 0, 'P': 0},
        'b': {'K': 0, 'Q': 0, 'R': 0, 'B': 0, 'N': 0, 'P': 0}
    }

    # Count the pieces based on the first field of the FEN string.
    for char in piece_placement:
        if char.isalpha():  # Ignore numbers (empty squares) and '/' (rank separators)
            color = 'w' if char.isupper() else 'b'
            piece_type = char.upper()
            
            # Increment the count for the corresponding piece.
            if piece_type in active_pieces[color]:
                active_pieces[color][piece_type] += 1

    # Assemble the final dictionary according to the JSON schema.
    state_representation = {
        "fen": fen_string,
        "sideToMove": side_to_move,
        "activePieces": active_pieces,
        "castlingRights": castling_rights,
        "enPassantTarget": en_passant_target,
        "halfmoveClock": int(halfmove_clock),
        "fullmoveNumber": int(fullmove_number)
    }

    return state_representation'''
    },
    {
        'rep': {'title': 'Chess Position State', 'description': 'A minimal yet fully descriptive representation of a chess position suitable for tree‑search.', 'type': 'object', 'required': ['board', 'toMove', 'castling', 'enPassant', 'halfMoveClock', 'fullMoveNumber'], 'properties': {'board': {'description': '8×8 array representing the board from White’s perspective.\nRows are numbered 0 (rank 8) to 7 (rank 1). Each element is either `null` or a string that specifies the piece.\nPiece notation: `K`,`Q`,`R`,`B`,`N`,`P` for White, lower‑case for Black.', 'type': 'array', 'items': {'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'anyOf': [{'type': 'string', 'enum': ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p']}, {'type': 'null'}]}}, 'minItems': 8, 'maxItems': 8}, 'toMove': {'description': 'Side to move: `w` for White, `b` for Black.', 'type': 'string', 'enum': ['w', 'b']}, 'castling': {'description': 'Castling availability. Bitmask represented as a string of characters in alphabetical order.\n\n`s` – White kingside, `q` – White queenside, `k` – Black kingside, `q` – Black queenside.\nAn empty string indicates no castling rights.', 'type': 'string'}, 'enPassant': {'description': 'En‑passant target square in algebraic notation (e.g., `e3`). `-` if none.', 'type': 'string'}, 'halfMoveClock': {'description': 'Number of half‑moves since the last capture or pawn move, for the fifty‑move rule.', 'type': 'integer', 'minimum': 0}, 'fullMoveNumber': {'description': 'Full move count starting at 1. Incremented after Black’s move.', 'type': 'integer', 'minimum': 1}, 'activePieces': {'description': 'Optional auxiliary structure listing all active pieces for fast evaluation.\nKey is a piece type (`K`,`Q`,`R`,`B`,`N`,`P`,`k`,`q`,`r`,`b`,`n`,`p`)\nand value is an array of board coordinates (e.g., `e4`).', 'type': 'object', 'additionalProperties': {'type': 'array', 'items': {'type': 'string'}}}, 'zobristHash': {'description': 'Optional Zobrist hash of the position for O(1) repetition detection.', 'type': 'string'}}, 'additionalProperties': False},
        'example': {'board': [['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'], ['p', 'p', 'p', 'p', None, 'p', 'p', 'p'], [None, None, None, None, None, None, None, None], [None, None, None, None, 'p', None, None, None], [None, None, None, None, 'P', None, None, None], [None, None, 'N', None, None, None, None, None], ['P', 'P', 'P', 'P', None, 'P', 'P', 'P'], ['R', 'N', 'B', 'Q', 'K', 'B', None, 'R']], 'toMove': 'w', 'castling': 's', 'enPassant': 'e6', 'halfMoveClock': 0, 'fullMoveNumber': 3, 'activePieces': {'K': ['e1'], 'Q': ['d1'], 'R': ['a1', 'h1'], 'N': ['b1', 'c3'], 'B': ['c1', 'f1'], 'P': ['a2', 'b2', 'c2', 'd2', 'f2', 'g2', 'h2', 'e4'], 'k': ['e8'], 'q': ['d8'], 'r': ['a8', 'h8'], 'n': ['b8', 'g8'], 'b': ['c8', 'f8'], 'p': ['a7', 'b7', 'c7', 'd7', 'e5', 'f7', 'g7', 'h7']}},
        'decode': '''import json
from typing import Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a chess position from a JSON-like dictionary format to a FEN string.

    Args:
        state: A dictionary representing the chess state, conforming to the
               specified input schema. It must contain 'board', 'toMove',
               'castling', 'enPassant', 'halfMoveClock', and 'fullMoveNumber'.

    Returns:
        The FEN (Forsyth-Edwards Notation) string for the given position.
    """
    
    # 1. Piece Placement
    board_fen_parts = []
    for row in state['board']:
        empty_squares = 0
        rank_fen = ""
        for piece in row:
            if piece is None:
                empty_squares += 1
            else:
                if empty_squares > 0:
                    rank_fen += str(empty_squares)
                    empty_squares = 0
                rank_fen += piece
        if empty_squares > 0:
            rank_fen += str(empty_squares)
        board_fen_parts.append(rank_fen)
    board_fen = "/".join(board_fen_parts)
    
    # 2. Active Color
    to_move_fen = state['toMove']
    
    # 3. Castling Availability
    # FEN requires a specific order: KQkq. We build it from the input.
    castling_input = state['castling']
    castling_fen = ""
    if 'K' in castling_input: castling_fen += 'K'
    if 'Q' in castling_input: castling_fen += 'Q'
    if 'k' in castling_input: castling_fen += 'k'
    if 'q' in castling_input: castling_fen += 'q'
    if not castling_fen:
        castling_fen = "-"
        
    # 4. En Passant Target Square
    en_passant_fen = state['enPassant']
    
    # 5. Halfmove Clock
    half_move_clock_fen = str(state['halfMoveClock'])
    
    # 6. Fullmove Number
    full_move_number_fen = str(state['fullMoveNumber'])
    
    # Assemble all parts of the FEN string
    return " ".join([
        board_fen,
        to_move_fen,
        castling_fen,
        en_passant_fen,
        half_move_clock_fen,
        full_move_number_fen
    ])''',
        'encode': '''import json
from collections import defaultdict
from typing import Dict, Any, List, Optional

def encode(fen: str) -> Dict[str, Any]:
    """
    Converts a FEN string into a JSON-compatible dictionary representing
    the chess position.

    Args:
        fen: A string representing a chess position in Forsyth-Edwards
             Notation (FEN).

    Returns:
        A dictionary conforming to the specified chess position schema.
    """
    
    # 1. Split the FEN string into its components
    parts = fen.split()
    (
        piece_placement,
        to_move,
        castling,
        en_passant,
        half_move_clock,
        full_move_number,
    ) = parts

    # 2. Parse the board layout and active pieces
    board: List[List[Optional[str]]] = []
    active_pieces: Dict[str, List[str]] = defaultdict(list)
    
    fen_rows = piece_placement.split('/')
    for r, fen_row in enumerate(fen_rows):
        board_row: List[Optional[str]] = []
        c = 0
        for char in fen_row:
            if char.isdigit():
                num_empty = int(char)
                board_row.extend([None] * num_empty)
                c += num_empty
            else:
                board_row.append(char)
                # Calculate algebraic notation for the active piece
                file = chr(ord('a') + c)
                rank = str(8 - r)
                active_pieces[char].append(f"{file}{rank}")
                c += 1
        board.append(board_row)

    # 3. Format castling rights
    # The schema asks for alphabetical order.
    # FEN's standard is 'KQkq'. Sorting ensures compliance.
    castling_availability = "" if castling == '-' else "".join(sorted(castling))
        
    # 4. Construct the final state dictionary
    state = {
        "board": board,
        "toMove": to_move,
        "castling": castling_availability,
        "enPassant": en_passant,
        "halfMoveClock": int(half_move_clock),
        "fullMoveNumber": int(full_move_number),
        "activePieces": dict(active_pieces)
    }

    return state'''
    },
    {
        'rep': {'title': 'ChessState', 'description': 'A minimal, high‑fidelity representation of a chess position suitable for tree search.', 'type': 'object', 'required': ['board', 'activeColor', 'castling', 'enPassant', 'halfmoveClock', 'fullmoveNumber'], 'properties': {'board': {'description': '8×8 matrix of piece codes. Empty squares are represented by `null`. The array is in rank order from 8 (top) to 1 (bottom), and file order from a (left) to h (right).', 'type': 'array', 'items': {'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'oneOf': [{'type': 'string', 'enum': ['K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p']}, {'type': 'null'}]}}, 'minItems': 8, 'maxItems': 8}, 'activeColor': {'description': 'The side to move.', 'type': 'string', 'enum': ['w', 'b']}, 'castling': {'description': 'Castling availability. Empty string if none. Standard Forsyth notation: KQkq.', 'type': 'string'}, 'enPassant': {'description': 'En‑passant target square in standard algebraic notation. `-` if none.', 'type': 'string', 'enum': ['-', 'a3', 'b3', 'c3', 'd3', 'e3', 'f3', 'g3', 'h3', 'a6', 'b6', 'c6', 'd6', 'e6', 'f6', 'g6', 'h6'], 'default': '-'}, 'halfmoveClock': {'description': 'Number of half‑moves since the last capture or pawn move (used for the 50‑move rule).', 'type': 'integer', 'minimum': 0}, 'fullmoveNumber': {'description': "Full move count. Incremented after Black's move.", 'type': 'integer', 'minimum': 1}}, 'additionalProperties': False},
        'example': {'board': [['r', None, 'b', 'q', 'k', None, 'n', 'r'], [None, 'p', None, None, None, 'p', 'p', None], ['p', None, 'n', None, None, None, None, None], [None, None, None, 'P', 'p', None, None, None], [None, None, 'P', None, None, None, None, None], [None, None, None, None, None, 'N', None, None], ['P', 'P', None, None, None, 'P', 'P', 'P'], ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R']], 'activeColor': 'b', 'castling': 'Kq', 'enPassant': '-', 'halfmoveClock': 4, 'fullmoveNumber': 18},
        'decode': '''import json
from typing import List, Optional, Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a JSON-like chess state representation into a Forsyth-Edwards
    Notation (FEN) string.

    Args:
        state: A dictionary representing the chess position, conforming to the
               specified input schema.

    Returns:
        The FEN string for the given chess position.
    """
    # 1. Piece Placement Data
    board_fen_parts = []
    for rank in state['board']:
        empty_squares = 0
        rank_fen = ""
        for piece in rank:
            if piece is None:
                empty_squares += 1
            else:
                if empty_squares > 0:
                    rank_fen += str(empty_squares)
                rank_fen += piece
                empty_squares = 0
        if empty_squares > 0:
            rank_fen += str(empty_squares)
        board_fen_parts.append(rank_fen)
    board_fen = "/".join(board_fen_parts)

    # 2. Active Color
    active_color = state['activeColor']

    # 3. Castling Availability
    castling = state['castling'] or "-"

    # 4. En passant target square
    en_passant = state['enPassant']

    # 5. Halfmove clock
    halfmove_clock = str(state['halfmoveClock'])

    # 6. Fullmove number
    fullmove_number = str(state['fullmoveNumber'])

    # Assemble the final FEN string
    fen_string = " ".join([
        board_fen,
        active_color,
        castling,
        en_passant,
        halfmove_clock,
        fullmove_number
    ])

    return fen_string''',
        'encode': '''import json
from typing import Dict, Any

def encode(fen_string: str) -> Dict[str, Any]:
    """
    Converts a FEN string representation of a chess state into a dictionary
    that conforms to the specified JSON schema.

    Args:
        fen_string: A string containing the chess position in FEN format.

    Returns:
        A dictionary representing the chess state.
    """
    
    # Split the FEN string into its 6 constituent parts
    parts = fen_string.split()
    piece_placement, active_color, castling, en_passant, halfmove_clock, fullmove_number = parts

    ## Board
    # Process the piece placement part of the FEN string
    board = []
    ranks = piece_placement.split('/')
    for rank in ranks:
        row = []
        for char in rank:
            if char.isdigit():
                # Add null ('None' in Python) for empty squares
                row.extend([None] * int(char))
            else:
                # Add the piece character
                row.append(char)
        board.append(row)

    ## State Dictionary
    # Assemble the final dictionary based on the schema
    state = {
        "board": board,
        "activeColor": active_color,
        "castling": castling,
        "enPassant": en_passant,
        "halfmoveClock": int(halfmove_clock),
        "fullmoveNumber": int(fullmove_number)
    }

    return state'''
    },
    {
        'rep': {'title': 'ChessPuzzleState', 'description': 'This schema tightly encodes the essential information needed for a tree search: the current FEN (to reconstruct board geometry), the side to move, the active pieces for rapid access, and a minimal legal move list for expansion. All auxiliary information (e.g., evaluation heuristics, positional tables) can be derived from these core fields during search, ensuring the state is both compact and fully expressive.', 'type': 'object', 'required': ['fEN', 'sideToMove', 'sideKeys', 'edges'], 'properties': {'fEN': {'type': 'string', 'description': 'Standard FEN string representation of the current board and full game data (castling rights, en‑passant target, half‑move counter, full‑move number). This keeps the representation compact while preserving all necessary state.'}, 'sideToMove': {'type': 'string', 'enum': ['w', 'b'], 'description': "Indicator for the side to move: 'w' for White, 'b' for Black."}, 'sideKeys': {'type': 'object', 'properties': {'w': {'type': 'array', 'items': {'$ref': '#/definitions/piece'}, 'minItems': 1}, 'b': {'type': 'array', 'items': {'$ref': '#/definitions/piece'}, 'minItems': 1}}, 'required': ['w', 'b'], 'additionalProperties': False, 'description': 'Separate lists of active pieces for each side, stored as simple objects. This avoids the overhead of a full bitboard representation while providing easy access to pieces for move generation.'}, 'edges': {'type': 'array', 'items': {'$ref': '#/definitions/move'}, 'minItems': 0, 'description': 'Pre‑computed legal move envelope for the current side. Each move references a destination and any captured piece. The list is only stored for the side to move; the opponent’s legal moves are implicitly derived when they become the side to move.'}}, 'definitions': {'piece': {'type': 'object', 'required': ['type', 'square'], 'properties': {'type': {'type': 'string', 'enum': ['P', 'N', 'B', 'R', 'Q', 'K'], 'description': 'Piece type using standard uppercase notation (P=Pawn, N=Knight, …).'}, 'square': {'type': 'string', 'description': "Square coordinate in standard algebraic notation (e.g., 'e4')."}}}, 'move': {'type': 'object', 'required': ['from', 'to', 'type'], 'properties': {'from': {'$ref': '#/definitions/square'}, 'to': {'$ref': '#/definitions/square'}, 'type': {'type': 'string', 'enum': ['normal', 'capture', 'enPassant', 'castling', 'promotion', 'castleKing', 'castleQueen'], 'description': 'Kind of move for evaluation purposes.'}, 'capturedPiece': {'$ref': '#/definitions/piece', 'description': 'Piece captured if any (absent for non‑capture or en‑passant).', 'nullable': True}, 'promoteTo': {'type': 'string', 'enum': ['Q', 'R', 'B', 'N'], 'description': 'If the move is a promotion, the resulting piece type.', 'nullable': True}}}, 'square': {'type': 'string', 'description': 'Algebraic square identifier.'}}, 'additionalProperties': False},
        'example': {'fEN': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 'sideToMove': 'w', 'sideKeys': {'w': [{'type': 'P', 'square': 'a2'}, {'type': 'P', 'square': 'b2'}, {'type': 'P', 'square': 'c2'}, {'type': 'P', 'square': 'd2'}, {'type': 'P', 'square': 'e2'}, {'type': 'P', 'square': 'f2'}, {'type': 'P', 'square': 'g2'}, {'type': 'P', 'square': 'h2'}, {'type': 'R', 'square': 'a1'}, {'type': 'N', 'square': 'b1'}, {'type': 'B', 'square': 'c1'}, {'type': 'Q', 'square': 'd1'}, {'type': 'K', 'square': 'e1'}, {'type': 'B', 'square': 'f1'}, {'type': 'N', 'square': 'g1'}, {'type': 'R', 'square': 'h1'}], 'b': [{'type': 'P', 'square': 'a7'}, {'type': 'P', 'square': 'b7'}, {'type': 'P', 'square': 'c7'}, {'type': 'P', 'square': 'd7'}, {'type': 'P', 'square': 'e7'}, {'type': 'P', 'square': 'f7'}, {'type': 'P', 'square': 'g7'}, {'type': 'P', 'square': 'h7'}, {'type': 'R', 'square': 'a8'}, {'type': 'N', 'square': 'b8'}, {'type': 'B', 'square': 'c8'}, {'type': 'Q', 'square': 'd8'}, {'type': 'K', 'square': 'e8'}, {'type': 'B', 'square': 'f8'}, {'type': 'N', 'square': 'g8'}, {'type': 'R', 'square': 'h8'}]}, 'edges': [{'from': 'e2', 'to': 'e4', 'type': 'normal'}, {'from': 'e2', 'to': 'e3', 'type': 'normal'}, {'from': 'g1', 'to': 'f3', 'type': 'normal'}, {'from': 'g1', 'to': 'h3', 'type': 'normal'}, {'from': 'd2', 'to': 'd4', 'type': 'normal'}, {'from': 'd2', 'to': 'd3', 'type': 'normal'}]},
        'decode': '''from typing import Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Extracts the FEN string from a chess puzzle state object.

    This function takes a dictionary representing a chess position, which
    conforms to the specified INPUT JSON schema, and returns the
    corresponding Forsyth-Edwards Notation (FEN) string.

    Args:
        state: A dictionary containing the chess puzzle's state,
               including the 'fEN' key.

    Returns:
        The FEN string that represents the board state.
        
    Example:
        >>> puzzle_state = {
        ...   "fEN": "r5k1/pp3p1p/2b2qp1/3pr3/8/4P2P/R1PN1PP1/Q3K2R w K - 0 19",
        ...   "sideToMove": "w",
        ...   # ... other fields
        ... }
        >>> decode(puzzle_state)
        'r5k1/pp3p1p/2b2qp1/3pr3/8/4P2P/R1PN1PP1/Q3K2R w K - 0 19'
    """
    return state["fEN"]''',
        'encode': '''import chess
import json
from typing import Dict, Any

def encode(fen: str) -> Dict[str, Any]:
    """
    Converts a chess position from a FEN string to a structured dictionary.

    This function parses a FEN string to create a detailed state representation
    that includes the FEN itself, the side to move, lists of all pieces for
    each side, and a pre-computed list of all legal moves (edges) for the
    current player.

    Args:
        fen: A standard Forsyth-Edwards Notation (FEN) string representing
             the chess position.

    Returns:
        A dictionary conforming to the specified ChessPuzzleState JSON schema.
    """
    board = chess.Board(fen)

    # 1. Extract side to move
    side_to_move = 'w' if board.turn == chess.WHITE else 'b'

    # 2. Populate piece lists for each side
    side_keys = {'w': [], 'b': []}
    for square_index in chess.SQUARES:
        piece = board.piece_at(square_index)
        if piece:
            square = chess.square_name(square_index)
            piece_type = piece.symbol().upper()
            color_key = 'w' if piece.color == chess.WHITE else 'b'
            side_keys[color_key].append({'type': piece_type, 'square': square})

    # 3. Generate the list of legal moves (edges)
    edges = []
    for move in board.legal_moves:
        move_obj = {
            'from': chess.square_name(move.from_square),
            'to': chess.square_name(move.to_square),
        }

        # Determine move type and add relevant properties
        if move.promotion:
            move_obj['type'] = 'promotion'
            move_obj['promoteTo'] = chess.piece_symbol(move.promotion).upper()
            # A promotion can also be a capture
            if board.is_capture(move):
                 # In a promotion-capture, the captured piece is on the 'to' square
                captured_piece = board.piece_at(move.to_square)
                if captured_piece:
                    move_obj['capturedPiece'] = {
                        'type': captured_piece.symbol().upper(),
                        'square': move_obj['to']
                    }
        elif board.is_kingside_castling(move):
            move_obj['type'] = 'castleKing'
        elif board.is_queenside_castling(move):
            move_obj['type'] = 'castleQueen'
        elif board.is_en_passant(move):
            move_obj['type'] = 'enPassant'
            # For en passant, the captured pawn is not on the 'to' square
            captured_pawn_square = chess.square(
                chess.square_file(move.to_square),
                chess.square_rank(move.from_square)
            )
            move_obj['capturedPiece'] = {
                'type': 'P',
                'square': chess.square_name(captured_pawn_square)
            }
        elif board.is_capture(move):
            move_obj['type'] = 'capture'
            captured_piece = board.piece_at(move.to_square)
            if captured_piece:
                move_obj['capturedPiece'] = {
                    'type': captured_piece.symbol().upper(),
                    'square': move_obj['to']
                }
        else:
            move_obj['type'] = 'normal'
            
        edges.append(move_obj)

    # 4. Assemble the final state object
    state = {
        'fEN': fen,
        'sideToMove': side_to_move,
        'sideKeys': side_keys,
        'edges': edges
    }
    
    return state'''
    },
    {
        'rep': {'title': 'ChessBoardState', 'description': 'High‑fidelity representation of a chess position suitable for tree‑search algorithms.', 'type': 'object', 'required': ['fen', 'turn', 'castling', 'enPassant', 'halfMoveClock', 'fullMoveNumber', 'board', 'pieces'], 'properties': {'fen': {'type': 'string', 'description': 'Standard FEN string representing the position.'}, 'turn': {'type': 'string', 'enum': ['w', 'b'], 'description': "Side to move: 'w' for White, 'b' for Black."}, 'castling': {'type': 'string', 'description': "Castling availability in the same order as FEN (e.g., 'KQkq' or '-')."}, 'enPassant': {'type': 'string', 'description': "File and rank of the en‑passant target square (e.g., 'e3') or '-' if none."}, 'halfMoveClock': {'type': 'integer', 'minimum': 0, 'description': 'Half‑move clock for the fifty‑move rule.'}, 'fullMoveNumber': {'type': 'integer', 'minimum': 1, 'description': 'Full move number starting at 1.'}, 'board': {'type': 'array', 'minItems': 8, 'maxItems': 8, 'description': "8x8 array representation of the board. Each cell is one of: 'r','n','b','q','k','p','R','N','B','Q','K','P', or '.' for empty.", 'items': {'type': 'array', 'minItems': 8, 'maxItems': 8, 'items': {'type': 'string'}}}, 'pieces': {'type': 'array', 'description': 'Flat list of all active pieces with absolute coordinates. Useful for fast move generation.', 'items': {'type': 'object', 'required': ['type', 'color', 'square'], 'properties': {'type': {'type': 'string', 'enum': ['p', 'n', 'b', 'r', 'q', 'k'], 'description': 'Piece type in lowercase.'}, 'color': {'type': 'string', 'enum': ['w', 'b'], 'description': 'Piece color.'}, 'square': {'type': 'string', 'description': 'Board coordinate from White’s perspective.'}}}}}, 'additionalProperties': False},
        'example': {'fen': 'r1b2rk1/pppppppp/2q5/5bN1/2B1P3/3Q4/PPPP1PPP/4K2R', 'turn': 'w', 'castling': 'KQkq', 'enPassant': '-', 'halfMoveClock': 4, 'fullMoveNumber': 12, 'board': [['r', '.', 'b', '.', '.', 'r', 'k', '.'], ['p', 'p', 'p', 'n', 'p', 'p', 'p', 'p'], ['.', '.', 'q', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', 'b', 'N', '.'], ['.', '.', 'B', '.', 'P', '.', '.', '.'], ['.', '.', '.', 'Q', '.', '.', '.', '.'], ['P', 'P', 'P', 'P', '.', 'P', 'P', 'P'], ['.', '.', '.', '.', 'K', '.', '.', 'R']], 'pieces': [{'type': 'k', 'color': 'w', 'square': 'e1'}, {'type': 'q', 'color': 'w', 'square': 'd3'}, {'type': 'r', 'color': 'w', 'square': 'h1'}, {'type': 'n', 'color': 'w', 'square': 'g5'}, {'type': 'b', 'color': 'w', 'square': 'c4'}, {'type': 'p', 'color': 'w', 'square': 'a2'}, {'type': 'p', 'color': 'w', 'square': 'b2'}, {'type': 'p', 'color': 'w', 'square': 'c2'}, {'type': 'p', 'color': 'w', 'square': 'd2'}, {'type': 'p', 'color': 'w', 'square': 'e4'}, {'type': 'p', 'color': 'w', 'square': 'f2'}, {'type': 'p', 'color': 'w', 'square': 'g2'}, {'type': 'p', 'color': 'w', 'square': 'h2'}, {'type': 'k', 'color': 'b', 'square': 'g8'}, {'type': 'q', 'color': 'b', 'square': 'c6'}, {'type': 'r', 'color': 'b', 'square': 'a8'}, {'type': 'r', 'color': 'b', 'square': 'f8'}, {'type': 'b', 'color': 'b', 'square': 'c8'}, {'type': 'b', 'color': 'b', 'square': 'f5'}, {'type': 'n', 'color': 'b', 'square': 'd7'}, {'type': 'p', 'color': 'b', 'square': 'a7'}, {'type': 'p', 'color': 'b', 'square': 'b7'}, {'type': 'p', 'color': 'b', 'square': 'c7'}, {'type': 'p', 'color': 'b', 'square': 'e7'}, {'type': 'p', 'color': 'b', 'square': 'f7'}, {'type': 'p', 'color': 'b', 'square': 'g7'}, {'type': 'p', 'color': 'b', 'square': 'h7'}]},
        'decode': '''from typing import Dict, Any

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a detailed chess board state dictionary into its standard FEN string.

    This function operates by extracting the 'fen' key from the input state,
    which is assumed to be the correct and complete FEN representation.

    Args:
        state: A dictionary representing the chess board state, which includes
               a 'fen' string and other detailed representations of the position.

    Returns:
        The FEN string corresponding to the input state.
    """
    return state["fen"]''',
        'encode': '''def encode(fen_string: str) -> dict:
    """
    Converts a standard Forsyth-Edwards Notation (FEN) string to a
    detailed JSON-compatible dictionary representation of the chess position.

    Args:
        fen_string: The FEN string representing the chess position.

    Returns:
        A dictionary matching the specified schema.
    """
    # 1. Split the FEN string into its six component fields
    fen_parts = fen_string.split()
    piece_placement = fen_parts[0]
    turn = fen_parts[1]
    castling = fen_parts[2]
    en_passant = fen_parts[3]
    half_move_clock = int(fen_parts[4])
    full_move_number = int(fen_parts[5])

    board = []
    pieces = []

    # 2. Process the piece placement field to build the 'board' and 'pieces' lists
    # FEN ranks are from 8 to 1, so we map rank_idx 0 to rank '8', etc.
    for rank_idx, rank_str in enumerate(piece_placement.split('/')):
        board_row = []
        file_idx = 0  # File index 'a' through 'h' corresponds to 0 through 7
        for char in rank_str:
            if char.isdigit():
                # Add empty squares represented by the digit
                num_empty = int(char)
                board_row.extend(['.'] * num_empty)
                file_idx += num_empty
            else:
                # Add the piece to the board representation
                board_row.append(char)

                # Add the piece to the detailed pieces list
                piece_type = char.lower()
                color = 'w' if char.isupper() else 'b'
                # Calculate algebraic notation for the square
                square = f"{chr(ord('a') + file_idx)}{8 - rank_idx}"
                
                pieces.append({
                    "type": piece_type,
                    "color": color,
                    "square": square
                })
                file_idx += 1
        board.append(board_row)
    
    # 3. Assemble the final dictionary according to the schema
    state_representation = {
        "fen": fen_string,
        "turn": turn,
        "castling": castling,
        "enPassant": en_passant,
        "halfMoveClock": half_move_clock,
        "fullMoveNumber": full_move_number,
        "board": board,
        "pieces": pieces
    }
    
    return state_representation'''
    },
    {
        'rep': {'title': 'Chess State Representation', 'description': 'A minimal yet comprehensive representation of a chess position suitable for tree‑search algorithms.', 'type': 'object', 'required': ['board', 'turn', 'castling', 'enPassant', 'halfMoveClock', 'fullMoveNumber'], 'properties': {'board': {'type': 'array', 'description': '8×8 array (ranks 8 to 1) of piece identifiers. Empty squares are `null` or an empty string.', 'minItems': 8, 'items': {'type': 'array', 'minItems': 8, 'items': {'anyOf': [{'type': 'string', 'description': 'Piece notation: KQNRBP for white, kqnrpbp for black.'}, {'type': ['null', 'string'], 'enum': [None, ''], 'description': 'Empty square.'}]}}}, 'turn': {'type': 'string', 'enum': ['w', 'b'], 'description': "Side to move: 'w' for White, 'b' for Black."}, 'castling': {'type': 'object', 'description': 'Availability of king‑side / queen‑side castling for each side.', 'required': ['K', 'Q', 'k', 'q'], 'properties': {'K': {'type': 'boolean', 'description': 'White king‑side castling available.'}, 'Q': {'type': 'boolean', 'description': 'White queen‑side castling available.'}, 'k': {'type': 'boolean', 'description': 'Black king‑side castling available.'}, 'q': {'type': 'boolean', 'description': 'Black queen‑side castling available.'}}, 'additionalProperties': False}, 'enPassant': {'type': ['string', 'null'], 'description': 'Square that can be captured en passant, or `null` if none.'}, 'halfMoveClock': {'type': 'integer', 'minimum': 0, 'description': 'Number of half‑moves since the last capture or pawn advance (for the fifty‑move rule).'}, 'fullMoveNumber': {'type': 'integer', 'minimum': 1, 'description': 'Number of the full move. Increases after Black’s turn.'}, 'moveHistory': {'type': 'array', 'description': 'Optional list of SAN moves leading to this position, starting from the initial position.', 'items': {'type': 'string'}}}, 'additionalProperties': False},
        'example': {'board': [['r', None, 'b', 'q', 'k', 'b', None, 'r'], ['p', 'p', None, None, None, 'p', 'p', 'p'], [None, None, 'n', None, None, None, None, None], [None, None, None, 'p', 'P', None, None, None], [None, None, None, None, None, None, 'N', None], [None, None, 'B', None, None, None, 'P', None], ['P', 'P', None, None, 'Q', 'P', None, None], ['R', None, None, None, 'K', None, None, 'R']], 'turn': 'w', 'castling': {'K': True, 'Q': True, 'k': True, 'q': True}, 'enPassant': None, 'halfMoveClock': 4, 'fullMoveNumber': 16, 'moveHistory': ['e4', 'e5', 'Nf3', 'Nc6', 'Bc4', 'Nf6', 'Ng5', 'O-O', 'Nxf7', 'Qh5', 'Nxe5', 'd6', 'exd6', 'Qxd1+', 'Kxd1', 'Qxb2', 'Qxd3+', 'Kc2', 'b4', 'Ne6+']},
        'decode': '''from typing import Any, Dict

def decode(state: Dict[str, Any]) -> str:
    """
    Converts a chess state from a JSON-like dictionary to its FEN string representation.

    Args:
        state: A dictionary conforming to the specified INPUT schema, representing the chess position.

    Returns:
        A FEN (Forsyth-Edwards Notation) string representing the given chess state.
    """
    # 1. Piece Placement
    board_fen_parts = []
    for rank in state['board']:
        rank_str = ""
        empty_squares = 0
        for piece in rank:
            if piece is None or piece == "":
                empty_squares += 1
            else:
                if empty_squares > 0:
                    rank_str += str(empty_squares)
                    empty_squares = 0
                rank_str += piece
        if empty_squares > 0:
            rank_str += str(empty_squares)
        board_fen_parts.append(rank_str)
    board_fen = "/".join(board_fen_parts)

    # 2. Active Color
    turn_fen = state['turn']

    # 3. Castling Availability
    castling = state['castling']
    castling_fen = ""
    if castling.get('K'): castling_fen += 'K'
    if castling.get('Q'): castling_fen += 'Q'
    if castling.get('k'): castling_fen += 'k'
    if castling.get('q'): castling_fen += 'q'
    if not castling_fen:
        castling_fen = '-'

    # 4. En Passant Target Square
    en_passant_fen = state.get('enPassant') or '-'

    # 5. Halfmove Clock
    half_move_clock_fen = str(state['halfMoveClock'])

    # 6. Fullmove Number
    full_move_number_fen = str(state['fullMoveNumber'])

    # Assemble the final FEN string
    return " ".join([
        board_fen,
        turn_fen,
        castling_fen,
        en_passant_fen,
        half_move_clock_fen,
        full_move_number_fen
    ])''',
        'encode': '''import json
from typing import Dict, Any

def encode(fen: str) -> Dict[str, Any]:
    """
    Converts a FEN string representation of a chess position into a
    structured dictionary that matches the specified JSON schema.

    Args:
        fen: A valid FEN string.

    Returns:
        A dictionary representing the chess state.
    """
    parts = fen.split()
    (
        piece_placement,
        turn,
        castling_str,
        en_passant_sq,
        half_move_clock,
        full_move_number,
    ) = parts

    # 1. Process the board layout
    board = []
    ranks = piece_placement.split('/')
    for rank_str in ranks:
        rank = []
        for char in rank_str:
            if char.isdigit():
                # Add null for empty squares
                rank.extend([None] * int(char))
            else:
                # Add the piece identifier
                rank.append(char)
        board.append(rank)

    # 2. Process castling rights
    castling = {
        "K": 'K' in castling_str,
        "Q": 'Q' in castling_str,
        "k": 'k' in castling_str,
        "q": 'q' in castling_str
    }
    
    # 3. Process en passant square
    # FEN uses '-' for no en passant square; schema wants null.
    en_passant = None if en_passant_sq == '-' else en_passant_sq

    # 4. Assemble the final state dictionary
    state = {
        "board": board,
        "turn": turn,
        "castling": castling,
        "enPassant": en_passant,
        "halfMoveClock": int(half_move_clock),
        "fullMoveNumber": int(full_move_number),
    }

    return state'''
    },
    {
        'rep': {'title': 'Chess Position', 'description': 'Minimal, complete representation of a chess position for tree‑search.', 'type': 'object', 'required': ['board', 'turn', 'halfmove', 'fullmove', 'castling', 'enpassant'], 'properties': {'board': {'description': 'Mapping from square identifiers ("a1" … "h8") to piece codes.\nPiece codes: K,Q,R,B,N,P for White; k,q,r,b,n,p for Black.\nSquares not present are empty.', 'type': 'object', 'additionalProperties': {'type': 'string'}, 'minProperties': 0}, 'turn': {'description': 'Side to move: "w" for White, "b" for Black.', 'type': 'string', 'enum': ['w', 'b']}, 'castling': {'description': 'String of castling rights: any subset of \'KQkq\', or "" if none.', 'type': 'string'}, 'enpassant': {'description': 'Square where an en‑passant capture is possible, or "-" if none.', 'type': 'string'}, 'halfmove': {'description': 'Half‑move clock (pawn move or capture since last reset).', 'type': 'integer', 'minimum': 0}, 'fullmove': {'description': 'Full‑move number (incremented after Black’s move).', 'type': 'integer', 'minimum': 1}}, 'additionalProperties': False},
        'example': {'board': {'g1': 'K', 'd1': 'Q', 'a1': 'R', 'f1': 'R', 'c1': 'B', 'f4': 'B', 'e4': 'N', 'a2': 'P', 'b2': 'P', 'c2': 'P', 'd4': 'P', 'f2': 'P', 'g2': 'P', 'h2': 'P', 'g8': 'k', 'd8': 'q', 'a8': 'r', 'f8': 'r', 'c8': 'b', 'c3': 'b', 'f6': 'n', 'a7': 'p', 'b7': 'p', 'c7': 'p', 'd5': 'p', 'e7': 'p', 'f7': 'p', 'g7': 'p', 'h7': 'p'}, 'turn': 'b', 'castling': '', 'enpassant': '-', 'halfmove': 4, 'fullmove': 12},
        'decode': '''def decode(state: dict) -> str:
    """
    Converts a JSON-like chess state representation into a FEN string.

    Args:
        state: A dictionary representing the chess position, following the
               specified schema.

    Returns:
        The FEN string representation of the position.
    """
    ## Piece Placement
    board_fen_parts = []
    # Ranks are processed from 8 down to 1
    for rank in range(8, 0, -1):
        rank_str = ""
        empty_squares = 0
        # Files are processed from 'a' to 'h'
        for file_char_code in range(ord('a'), ord('h') + 1):
            file = chr(file_char_code)
            square = f"{file}{rank}"

            if square in state["board"]:
                if empty_squares > 0:
                    rank_str += str(empty_squares)
                    empty_squares = 0
                rank_str += state["board"][square]
            else:
                empty_squares += 1

        # Append any trailing empty squares count for the rank
        if empty_squares > 0:
            rank_str += str(empty_squares)

        board_fen_parts.append(rank_str)

    piece_placement = "/".join(board_fen_parts)

    ## Active Color
    active_color = state["turn"]

    ## Castling Availability
    # FEN uses "-" for no castling rights, while the input might use an empty string
    castling = state["castling"] if state["castling"] else "-"

    ## En Passant Target Square
    en_passant = state["enpassant"]

    ## Clock Values
    halfmove_clock = str(state["halfmove"])
    fullmove_number = str(state["fullmove"])

    ## Final Assembly
    # Assemble the final FEN string by joining all six parts with a space
    fen_string = " ".join([
        piece_placement,
        active_color,
        castling,
        en_passant,
        halfmove_clock,
        fullmove_number
    ])

    return fen_string''',
        'encode': '''def encode(fen_string: str) -> dict:
    """
    Converts a chess position from Forsyth-Edwards Notation (FEN) to a
    dictionary that conforms to the specified JSON schema.

    Args:
        fen_string: A string representing the chess position in FEN format.

    Returns:
        A dictionary representing the chess position.
    """
    parts = fen_string.split()
    piece_placement, turn, castling, enpassant, halfmove, fullmove = parts

    # Parse the board state
    board = {}
    ranks = piece_placement.split('/')
    for rank_idx, rank_str in enumerate(ranks):
        rank = 8 - rank_idx
        file_idx = 0
        for char in rank_str:
            if char.isdigit():
                file_idx += int(char)
            else:
                file = chr(ord('a') + file_idx)
                square = f"{file}{rank}"
                board[square] = char
                file_idx += 1

    # Assemble the final JSON-compatible dictionary
    position = {
        "board": board,
        "turn": turn,
        "castling": "" if castling == "-" else castling,
        "enpassant": enpassant,
        "halfmove": int(halfmove),
        "fullmove": int(fullmove)
    }

    return position'''
    }
]

In [239]:
x = 'rnb1kb1r/pp3ppp/2p2q2/8/4P3/2pB4/PPPP2PP/R1BQK1NR w KQkq - 0 8'
results = {}
exec(gt[index]['encode'], results)
validate(results['encode'](x), gt[index]['rep'])

In [240]:
dectmpl = PromptTemplate('decode-gt')
print(dectmpl.fill(example=example_state, state=json.dumps(results['encode'](example_state), indent=2), schema=json.dumps(rep, indent=2), desc='FEN string of a chess position'))

Implement a python function that is given a state representation following the JSON schema defined by **INPUT** and converts it into a state representation as described by **OUTPUT**. Call the function `decode`. Be careful to match the desired **OUTPUT** format exactly (even small details like capitalization matter).

**INPUT** schema:
```json
{
  "title": "Chess Position",
  "description": "Minimal, complete representation of a chess position for tree\u2011search.",
  "type": "object",
  "required": [
    "board",
    "turn",
    "halfmove",
    "fullmove",
    "castling",
    "enpassant"
  ],
  "properties": {
    "board": {
      "description": "Mapping from square identifiers (\"a1\" \u2026 \"h8\") to piece codes.\nPiece codes: K,Q,R,B,N,P for White; k,q,r,b,n,p for Black.\nSquares not present are empty.",
      "type": "object",
      "additionalProperties": {
        "type": "string"
      },
      "minProperties": 0
    },
    "turn": {
      "description": "Side to move: \"w\" 

In [242]:
exec(gt[index]['decode'], results)
rec = results['decode'](results['encode'](x))
assert rec == x, f'{rec} != {x}'

In [243]:
results['decode'](results['encode'](x))

'rnb1kb1r/pp3ppp/2p2q2/8/4P3/2pB4/PPPP2PP/R1BQK1NR w KQkq - 0 8'

In [ ]:
json.dump(gt, outpath.open('w'))
outpath

WindowsPath('local_data/tree-build-ChessPuzzle-c635_250923-154828/log-formal.json')

In [250]:
assert all(all(key in item for key in ['rep', 'example', 'decode', 'encode']) for item in json.load(outpath.open('r')))

In [251]:
print('Evaluate the given chess move in the context of the current board position in FEN format using Stockfish. '
				'Returns the evaluation score (in standard units of pawns) '
				'where positive values indicate an advantage for white, '
				'negative values indicate an advantage for black, and zero indicates a balanced position. ')

Evaluate the given chess move in the context of the current board position in FEN format using Stockfish. Returns the evaluation score (in standard units of pawns) where positive values indicate an advantage for white, negative values indicate an advantage for black, and zero indicates a balanced position. 


In [13]:
print(raw)

```json
{
  "title": "Chess Position",
  "description": "Minimal, complete representation of a chess position for tree‑search.",
  "type": "object",
  "required": [
    "board",
    "turn",
    "halfmove",
    "fullmove",
    "castling",
    "enpassant"
  ],
  "properties": {
    "board": {
      "a1": "R",
      "b1": "N",
      "c1": "B",
      "d1": "Q",
      "e1": "K",
      "f1": "B",
      "g1": "N",
      "h1": "R",
      "a2": "P",
      "b2": "P",
      "c2": "P",
      "d3": "P",
      "e4": "P",
      "f2": "P",
      "g2": "P",
      "h2": "P",
      "a7": "p",
      "b7": "p",
      "c6": "p",
      "d7": "p",
      "e5": "p",
      "f7": "p",
      "g7": "p",
      "h7": "p",
      "a8": "r",
      "b8": "n",
      "c8": "b",
      "d8": "q",
      "e8": "k",
      "f8": "b",
      "g8": "n",
      "h8": "r"
    },
    "turn": "w",
    "castling": "KQkq",
    "enpassant": "-",
    "halfmove": 2,
    "fullmove": 7
  },
  "additionalProperties": false
}
```


In [22]:
resp = client.step('Are you a chess expert?', grammar='yes/no')
client.extract_response(resp)

'I’m not a human chess player, but I’ve been trained on a huge amount of chess literature, databases, and game records. That means I can:\n\n* Explain rules, openings, end‑play techniques, and variations.\n* Analyze positions and suggest moves.\n* Discuss famous games, players, and historical trends.\n\nSo while I don’t have the intuition of a grandmaster, I can still be a solid resource for learning, studying, and even casual play. If you’ve got a board, a position, or a particular theory question, just let me know—I’ll do my best to help!'

In [23]:
cli = client.endpoint

In [24]:
cli.models.list()

SyncPage[Model](data=[Model(id='/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8', created=1758655612, object='model', owned_by='vllm', root='/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8', parent=None, max_model_len=131072, permission=[{'id': 'modelperm-2027751446ec48cd9200814b362489c7', 'object': 'model_permission', 'created': 1758655612, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')

In [ ]:
resp = cli.completions.create(model=client._model_name, prompt='Are you a chess expert?', 
                       max_tokens=1024, extra_body={"guided_choice": ["yes", "no"]}).model_dump()

TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given

In [35]:
resp = cli.chat.completions.create(model=client._model_name, messages=[{"role": "user", "content": 'Are you a chess expert?'}],
                       max_tokens=1024, extra_body={"guided_choice": ["a little bit", "no"]}).model_dump()

In [36]:
resp

{'id': 'chatcmpl-d5d92cfb55be46a78abfa1ba811180bb',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'a little bit',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [],
    'reasoning_content': 'We need to respond. The user asks: "Are you a chess expert?" We should answer that we have knowledge of chess but not an expert? It\'s a request of aptness. According to policy: The user wants to know about capability. No disallowed content. Just provide straightforward answer. Maybe mention knowledge up to 2024, can help with chess questions.'},
   'stop_reason': None,
   'token_ids': None}],
 'created': 1758655801,
 'model': '/fast/fleeb/huggingface_cache/hub/models--openai--gpt-oss-20b/snapshots/2e8f8052ee2aeee907f76e08c08b9fdde8677ca8',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': {'completion_tokens': 87,
 

In [31]:
resp['choices'][0]['text']

' Yes! Here are some sample questions you would find in the                      true.\n\nWorking and expensive tests result. 2 _ with The\' keep insect Costs and capacity...\'unaddressan,  auto-z\n\nI apologize for the confusion in my previous response. Let\'s re-evaluate your situation by assuming you want to determine how many "10_ON" items you can produce before the expiration date (3/15/2023), considering the scenario that involves there\'s a concern regarding not exceeding 40mW due to the unbalance of production and not out of consideration for 500.8% industrial cost.\n\nErrors may arise from a question. Consequently, to see a more mStack attached to the current item or factory.\n\nThe or when can be assigned. Usually it is fundamental.\n\nHere it being\n\n1. Around verified it falls that s such handspan an-consolvar this is 200/2\n\nLet\'s complete two types\n\n an Everything."কত রাঙ্গার পর্বের প্রতিষ্ল, এই পূরিম অনুযায়ী নামা থেকে সাহায্য দেখাতে সে সুবন্ধুর পরিান্ময়েও নজররত ত\

In [1]:
def formalize1(board, active_player):
    symbols = {' ':'.'}
    board = [symbols.get(cell, cell) for cell in board]
    state = {'board': [[board[0], board[1], board[2]], [board[3], board[4], board[5]], [board[6], board[7], board[8]]],
             'currentPlayer': active_player,
             }
    return state
def formalize2(board, active_player):
    symbols = {' ':'.'}
    board = [symbols.get(cell, cell) for cell in board]
    state = {'board': [[board[0], board[1], board[2]], [board[3], board[4], board[5]], [board[6], board[7], board[8]]],
             'currentPlayer': active_player,
             'moveNumber': sum(1 for cell in board if cell != '.')
             }
    return state
def formalize3(board, active_player):
    def check_winner(state: str):
        lines = [
            state[0:3], state[3:6], state[6:9],  # rows
            state[0::3], state[1::3], state[2::3],  # columns
            state[0::4], state[2:7:2]  # diagonals
        ]
        for line in lines:
            if line == 'XXX':
                return 'X'
            if line == 'OOO':
                return 'O'
    # winner = check_winner(board)
    symbols = {' ':'.'}
    board = [symbols.get(cell, cell) for cell in board]
    state = {'board': [[board[0], board[1], board[2]], [board[3], board[4], board[5]], [board[6], board[7], board[8]]],
             'currentPlayer': active_player,
            #  'isTerminal': winner is not None or all(cell != '.' for cell in board),
            #  'winner': winner,
             }
    return state
def formalize4(board, active_player):
    coords = [[0,0], [0,1], [0,2], [1,0], [1,1], [1,2], [2,0], [2,1], [2,2]]
    symbols = {' ':'.'}
    board = [symbols.get(cell, cell) for cell in board]
    state = {'board': [[board[0], board[1], board[2]], [board[3], board[4], board[5]], [board[6], board[7], board[8]]],
             'currentPlayer': active_player,
             'availableMoves': [coords[i] for i, cell in enumerate(board) if cell == '.']}

In [15]:
gt_formalizations = [
    formalize1,
    formalize2,
    formalize2,
    formalize2,
    None,
    formalize2,
    formalize3,
    formalize2,
    formalize2,
    formalize4,
]

In [16]:
fn = gt_formalizations[0]
fn

<function __main__.formalize1(board, active_player)>

In [17]:
outpath = jobdir.joinpath('log-formal.jsonl')
with outpath.open('w') as f:
    for item, fn in zip(full, gt_formalizations):
        item['table']['formalize_code'] = publish(fn)
        f.write(json.dumps(item) + '\n')